In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os, glob
 
data_dir = './test'  # train
features_dir = './Vgg_features_test'  # Vgg_features_train
 
 
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output
 
 
model = Encoder()
model = model.cuda()
 
 
def extractor(img_path, saved_path, net, use_gpu):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()]
    )
 
    img = Image.open(img_path)
    img = transform(img)
    print(img.shape)
 
    x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
    print(x.shape)
 
    if use_gpu:
        x = x.cuda()
        net = net.cuda()
    y = net(x).cpu()
    y = torch.squeeze(y)
    y = y.data.numpy()
    print(y.shape)
    np.savetxt(saved_path, y, delimiter=',')
 
 
# extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']

files_list = []
x = os.walk(data_dir)
for path, d, filelist in x:
    for filename in filelist:
        file_glob = os.path.join(path, filename)
        files_list.extend(glob.glob(file_glob))

print(files_list)

use_gpu = torch.cuda.is_available()

for x_path in files_list:
    print("x_path" + x_path)
    file_name = x_path.split('/')[-1]
    fx_path = os.path.join(features_dir, file_name + '.txt')
    print(fx_path)
    extractor(x_path, fx_path, model, use_gpu)

['./test/a.png', './test/x.png', './test/enhanced_a.png']
x_path./test/a.png
./Vgg_features_test/a.png.txt
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path./test/x.png
./Vgg_features_test/x.png.txt
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path./test/enhanced_a.png
./Vgg_features_test/enhanced_a.png.txt
torch.Size([4, 224, 224])
torch.Size([1, 4, 224, 224])


/mistgpu/miniconda/envs/lxj/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 4, 224, 224] to have 3 channels, but got 4 channels instead

In [4]:
# 目标是改装成一次加载，之后处理单张图片
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os, glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class VGGEncoder(nn.Module):
    def __init__(self):
        super(VGGEncoder, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output
 
VGG_model = VGGEncoder().to(device)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()]
)




In [2]:
img = Image.open('./test/a.png')

img = transform(img)
x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False).to(device)
y = VGG_model(x).cpu()
y = torch.squeeze(y)
# y = y.data.numpy()

NameError: name 'Image' is not defined

In [1]:
img = Image.open('./test/x.png')
img = transform(img)
xx = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False).to(device)
yy = VGG_model(xx).cpu()
yy = torch.squeeze(yy)
print(yy)
# yy = yy.data.numpy()

NameError: name 'Image' is not defined

In [6]:
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import torch


model = models.vgg16(pretrained=True)
model=model.eval()	# 一定要有这行，不然运算速度会变慢（要求梯度）而且会影响结果
model.cuda()	# 将模型从CPU发送到GPU,如果没有GPU则删除该行
TARGET_IMG_SIZE = 224
img_to_tensor = transforms.ToTensor()

imgpath = './test/x.png'
img=Image.open(imgpath)		# 读取图片
img=img.resize((TARGET_IMG_SIZE, TARGET_IMG_SIZE))
x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False).to(device)
   
tensor= img_to_tensor(x)	# 将图片转化成tensor
tensor=tensor.cuda()	# 如果只是在cpu上跑的话要将这行去掉

result=model(Variable(tensor))
result
# result_npy=result.data.numpy()	# 保存的时候一定要记得转成cpu形式的，不然可能会出错

TypeError: unsqueeze(): argument 'input' (position 1) must be Tensor, not Image